<a href="https://colab.research.google.com/github/vipashaaV321/Collaborative-Food-Recipe-Recommendation-System/blob/Sarvesh/Data_preprocessing4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [ ]:
Base_path ='/content/drive/MyDrive/RS Data/Assignment 2'

In [ ]:
raw_recipies = pd.read_csv(Base_path+"/RAW_recipes.csv")
int_train = pd.read_csv(Base_path+"/interactions_train.csv")
int_val = pd.read_csv(Base_path+"/interactions_validation.csv")
int_test = pd.read_csv(Base_path+"/interactions_test.csv")

In [ ]:
user_counts = int_train.groupby('user_id').count()

# filter the user counts DataFrame to include only users who have rated at least 20 items
user_counts_filtered = user_counts[user_counts['recipe_id'] >= 100]

# get the list of user IDs who have rated at least 20 items
users_with_20_items = user_counts_filtered.index.tolist()

# filter the original DataFrame to include only the users who have rated at least 20 items
df_filtered = int_train[int_train['user_id'].isin(users_with_20_items)]

# print the filtered DataFrame
print(df_filtered)

           user_id  recipe_id        date  rating     u       i
6             2312        780  2000-09-12     5.0  1674  127175
7             2312      51964  2000-09-26     5.0  1674  151793
8             2312       1232  2000-10-17     4.0  1674   15498
9             2312       4397  2000-10-17     5.0  1674   14380
11            2312        164  2000-10-20     5.0  1674   96573
...            ...        ...         ...     ...   ...     ...
698887      146742     278872  2018-12-16     5.0  2178   76230
698888      400708       5123  2018-12-17     5.0   560   72083
698892     1271506      53121  2018-12-17     5.0  2877   51525
698899      226867     363072  2018-12-18     5.0  3604   29101
698900  2000498330     314535  2018-12-19     5.0  2248   21514

[410249 rows x 6 columns]


In [ ]:
user_item_matrix = df_filtered.pivot_table(values='rating', index='user_id', columns='recipe_id')

# fill missing values with 0
user_item_matrix = user_item_matrix.fillna(0)

# print the user-item rating matrix
print(user_item_matrix)

recipe_id   38      40      45      46      49      52      58      62      \
user_id                                                                      
1533           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1535           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2310           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2312           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3288           0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...            ...     ...     ...     ...     ...     ...     ...     ...   
1800054678     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1802849661     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2000431901     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2000498330     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2001102678     0.0     0.0     0.0     0.0     0.0     0.0     0

In [ ]:
users = user_item_matrix.index.tolist()

In [ ]:
normalised_mat = user_item_matrix - np.asarray([(np.mean(user_item_matrix, 1))]).T

In [ ]:
A = normalised_mat.T / np.sqrt(user_item_matrix.shape[0] - 1)

In [ ]:
print(A)

user_id    1533        1535        2310        2312        3288        \
recipe_id                                                               
38          -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
40          -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
45          -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
46          -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
49          -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
...               ...         ...         ...         ...         ...   
536729      -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
536990      -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
537175      -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
537319      -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   
537458      -0.000112   -0.000598   -0.000327   -0.000144   -0.000156   

user_id    4291        4439        4470        474

In [ ]:
U, S, V = np.linalg.svd(A,full_matrices=False)